In [1]:
# put columns that need to be dropped
 

# Information value cut off
iv_co = 0.05  # 0.1


# Correlation cut off
cr_co = 0.4

# Max number of categorical bins
max_cat_bins = 3

# Min number of records to consider as `ELSE` group
min_num = 50

# Numerical bins (do not change if not really needed)
max_bin = 5 # 12  # 5
force_bin = 3 

################## Binning ######################
def mono_bin(Y, X, n = max_bin):
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1: # < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
    d3["IV"] = (d3.NON_EVENT_RATE-d3.EVENT_RATE)*np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME', 'MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return d3

def char_bin(Y, X, max_cat_bins = max_cat_bins, name = []):
    df1 = pd.DataFrame({"X": X, "Y": Y})
    df2 = df1.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.groups
    d3["MAX_VALUE"] = df2.groups
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y     
    d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
    d3["IV"] = (d3.NON_EVENT_RATE-d3.EVENT_RATE)*np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE',  'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.sort_values(by='WOE')
    d3 = d3.reset_index(drop=True)
    
    mn = 10
    min_x = np.nan
    min_y = np.nan
    for i in range(len(d3)):
        for j in range(len(d3)):
            if abs(d3.WOE[i]-d3.WOE[j])<mn and i!=j:
                mn = abs(d3.WOE[i]-d3.WOE[j])
                min_x = i
                min_y = j

    # replace
 
    try:
        val_to_rep = d3.MIN_VALUE[min_x] +' | '+ d3.MIN_VALUE[min_y]
    except:
        print (X.name)
        
    if len(d3)>max_cat_bins:
        X.replace(d3.MIN_VALUE[min_x], val_to_rep, inplace=True)
        X.replace(d3.MIN_VALUE[min_y], val_to_rep, inplace=True)
        # df_train[name].replace(d3.MIN_VALUE[min_x], val_to_rep, inplace=True)
        # df_train[name].replace(d3.MIN_VALUE[min_y], val_to_rep, inplace=True)
        # df_test[name].replace(d3.MIN_VALUE[min_x], val_to_rep, inplace=True)
        # df_test[name].replace(d3.MIN_VALUE[min_y], val_to_rep, inplace=True)
    
    while len(d3)>max_cat_bins:
        df1 = pd.DataFrame({"X": X, "Y": Y})
        df2 = df1.groupby('X',as_index=True)

        d3 = pd.DataFrame({},index=[])
        d3["COUNT"] = df2.count().Y
        d3["MIN_VALUE"] = df2.groups
        d3["MAX_VALUE"] = df2.groups
        d3["EVENT"] = df2.sum().Y
        d3["NONEVENT"] = df2.count().Y - df2.sum().Y     
        d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
        d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
        d3["WOE"] = np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
        d3["IV"] = (d3.NON_EVENT_RATE-d3.EVENT_RATE)*np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
        d3["VAR_NAME"] = "VAR"
        d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE',  'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
        d3 = d3.replace([np.inf, -np.inf], 0)
        d3.IV = d3.IV.sum()
        d3 = d3.sort_values(by='WOE')
        d3 = d3.reset_index(drop=True)

        mn = 10
        min_x = np.nan
        min_y = np.nan
        for i in range(len(d3)):
            for j in range(len(d3)):
                if abs(d3.WOE[i]-d3.WOE[j])<mn and i!=j:
                    mn = abs(d3.WOE[i]-d3.WOE[j])
                    min_x = i
                    min_y = j

        # replace
        if len(d3)>max_cat_bins:    
            val_to_rep = d3.MIN_VALUE[min_x] +' | '+ d3.MIN_VALUE[min_y]
            X.replace(d3.MIN_VALUE[min_x], val_to_rep, inplace=True)
            X.replace(d3.MIN_VALUE[min_y], val_to_rep, inplace=True)
            # df_train[name].replace(d3.MIN_VALUE[min_x], val_to_rep, inplace=True)
            # df_train[name].replace(d3.MIN_VALUE[min_y], val_to_rep, inplace=True)
            # df_test[name].replace(d3.MIN_VALUE[min_x], val_to_rep, inplace=True)
            # df_test[name].replace(d3.MIN_VALUE[min_y], val_to_rep, inplace=True)
    
    return d3

def char_bin_else(Y, X, name = []):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    df2 = df1.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.groups
    d3["MAX_VALUE"] = df2.groups
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y     
    d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
    d3["IV"] = (d3.NON_EVENT_RATE-d3.EVENT_RATE)*np.log(d3.NON_EVENT_RATE/d3.EVENT_RATE)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE',  'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return d3

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
   
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(pd.Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            elif df1[i].dtype==object:
                print(i)
                conv = char_bin(target, df1[i], name = i)
                conv["VAR_NAME"] = i            
                count = count + 1
            else:
                conv = char_bin_else(target, df1[i], name = i)
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    iv = iv.sort_values(by='IV', ascending=False)
    iv_df = iv_df[iv_df.COUNT!=0]
    return (iv_df, iv)

def woe_transformation(x, iv_col):
    if isinstance(x, str):
        q = float(iv_col[iv_col['MIN_VALUE'] == x].WOE)
        return q
    elif pd.isna(x)==True:
        q = float(iv_col[pd.isna(iv_col.MAX_VALUE)==True].WOE)
        return q

    if x>iv_col.MAX_VALUE.max():
        q = float(iv_col[iv_col.MAX_VALUE == iv_col.MAX_VALUE.max()].WOE)
    elif x<iv_col.MAX_VALUE.min():
        q = float(iv_col.head(1).WOE)
    # elif pd.isna(x)==True:
    #     q = float(iv_col[pd.isna(iv_col.MAX_VALUE)==True].WOE)
    else:
        # try:
        q = float(iv_col[(x>=iv_col.MIN_VALUE) & (x<=iv_col.MAX_VALUE)].WOE)
        # except:
            # q = float(iv_col[((iv_col.MAX_VALUE - x).abs().argsort()==0)].WOE)
    return q

def transform_df_to_woe(df, y, IV, iv_df, iv_cut_off=iv_co):
    
    df1 = df[list(IV[IV.IV>iv_cut_off].VAR_NAME)]
    try:
        df1 = df1.drop('credit_id', axis=1)
    except:
        pass
    try:
        df1 = df1.drop('date_requested', axis=1)
    except:
        pass
    print('Features left after IV drop: {}'.format(len(df1.columns)))
    for col in tqdm_notebook(df1.columns):
        iv_col = iv_df[iv_df.VAR_NAME==col]
        try:
            df1[col] = df1[col].map(lambda x: woe_transformation(x, iv_col))
        except Exception as e:
            print (col, f'- ERROR! Column was dropped!, Exception={e}')
            df1.drop(col, axis=1, inplace=True)
    return df1, y